In [105]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import requests 
import json
import os
from dotenv import load_dotenv
import time
import sys
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append("../")

In [106]:
#url = "https://api.spacexdata.com/v4/starlink"
#url = "https://api.wheretheiss.at/v1/satellites/25544" 
url = "http://api.open-notify.org/iss-now.json"
url2 = "https://satellitemap.space/json/sl.json?0.2592483420529641+v1007+19253"

In [107]:
req = requests.get(url)
req.content

b'{"message": "success", "timestamp": 1682949869, "iss_position": {"longitude": "173.8488", "latitude": "41.6243"}}'

In [108]:
req2 = requests.get(url2)
req2.content #1682930043

b'{"stamp":1682949844,"sats":[{"id":25544,"name":"0-A","oname":"ISS","lat":40.8785,"lng":172.1645,"alt":423,"alt2":425.8,"p":414,"lat2":49.6051,"lng2":-164.3842,"illum":0,"raan":200.7111,"age":0.2},{"id":44713,"name":"2-A","oname":"STARLINK 1007","lat":3.0644,"lng":-63.2595,"alt":547.3,"alt2":550.3,"p":546.4,"lat2":-12.0290,"lng2":-53.0624,"illum":1,"raan":194.1989,"age":1},{"id":44714,"name":"2-B","oname":"STARLINK 1008","lat":-48.5021,"lng":61.3151,"alt":564.7,"alt2":560.3,"p":546.5,"lat2":-38.1691,"lng2":81.7607,"illum":1,"raan":194.3001,"age":1.1},{"id":44715,"name":"2-C","oname":"STARLINK 1009","lat":-7.4826,"lng":113.4023,"alt":549.6,"alt2":547.4,"p":546.7,"lat2":7.6400,"lng2":123.5292,"illum":0,"raan":193.7521,"age":0.9},{"id":44716,"name":"2-D","oname":"STARLINK 1010","lat":-49.1646,"lng":-1.0200,"alt":564.8,"alt2":566.5,"p":546.9,"lat2":-53.2020,"lng2":27.1265,"illum":1,"raan":192.4803,"age":0.7},{"id":44717,"name":"2-E","oname":"STARLINK 1011","lat":-52.2442,"lng":34.0141,"al

In [109]:
req2.json()['sats'][1:]

[{'id': 44713,
  'name': '2-A',
  'oname': 'STARLINK 1007',
  'lat': 3.0644,
  'lng': -63.2595,
  'alt': 547.3,
  'alt2': 550.3,
  'p': 546.4,
  'lat2': -12.029,
  'lng2': -53.0624,
  'illum': 1,
  'raan': 194.1989,
  'age': 1},
 {'id': 44714,
  'name': '2-B',
  'oname': 'STARLINK 1008',
  'lat': -48.5021,
  'lng': 61.3151,
  'alt': 564.7,
  'alt2': 560.3,
  'p': 546.5,
  'lat2': -38.1691,
  'lng2': 81.7607,
  'illum': 1,
  'raan': 194.3001,
  'age': 1.1},
 {'id': 44715,
  'name': '2-C',
  'oname': 'STARLINK 1009',
  'lat': -7.4826,
  'lng': 113.4023,
  'alt': 549.6,
  'alt2': 547.4,
  'p': 546.7,
  'lat2': 7.64,
  'lng2': 123.5292,
  'illum': 0,
  'raan': 193.7521,
  'age': 0.9},
 {'id': 44716,
  'name': '2-D',
  'oname': 'STARLINK 1010',
  'lat': -49.1646,
  'lng': -1.02,
  'alt': 564.8,
  'alt2': 566.5,
  'p': 546.9,
  'lat2': -53.202,
  'lng2': 27.1265,
  'illum': 1,
  'raan': 192.4803,
  'age': 0.7},
 {'id': 44717,
  'name': '2-E',
  'oname': 'STARLINK 1011',
  'lat': -52.2442,
  

In [110]:
df_sl = pd.DataFrame(requests.get(url2).json()['sats'][1:])
df_sl = df_sl[df_sl['oname'] != 'noname']
df_sl = df_sl[['name', 'oname', 'lat', 'lng']]
df_sl

,name,oname,lat,lng
0,2-A,STARLINK 1007,3.0644,-63.2595
1,2-B,STARLINK 1008,-48.5021,61.3151
2,2-C,STARLINK 1009,-7.4826,113.4023
3,2-D,STARLINK 1010,-49.1646,-1.0200
4,2-E,STARLINK 1011,-52.2442,34.0141
...,...,...,...,...
3834,79-AZ,STARLINK-5758,36.5817,-52.7400
3835,79-BA,STARLINK-5793,19.1059,-83.2979
3836,79-BB,STARLINK-5795,36.9453,-51.7414
3837,79-BC,STARLINK-5798,37.0559,-51.4371


In [93]:
req.json()['iss_position']

{'longitude': '164.3117', 'latitude': '35.5990'}

In [94]:
speed_f9 = 6724
height_iss = 400000
earth_radius = 40008
speed_iss = 30000

distance = earth_radius + height_iss
relative_speed = speed_f9 + speed_iss

time = distance / relative_speed

minutes, seconds = divmod(time*60, 60)

print(f"The Falcon 9 and the ISS will cross in {int(minutes)} mins and {int(seconds)} segs.")

The Falcon 9 and the ISS will cross in 11 mins and 58 segs.


In [95]:
time_iss = (11*60)+58
iss_distance = speed_iss / time_iss
print(f'The StarLink will travel {iss_distance:.2f}km in 11 min and 58 segs')

The StarLink will travel 41.78km in 11 min and 58 segs


In [119]:
iss_location = []
location_iss = requests.get(url).json()
location_sl = requests.get(url2).json()['sats'][1:]
position_sl = []
for i in range(5):
    
    if location_iss['message'] == 'success':
        time.sleep(2)
        print(i)
        #location_iss = requests.get(url).json()
        iss_location.append(location_iss['iss_position'])
    if 'sats' in location_sl:
        time.sleep(2)
        print(i)
        position_sl.append(location_sl)
        #location_sl = requests.get(url2).json()['sats'][1:]
    else:
        break
        
#iss_location
df_iss = pd.DataFrame(iss_location)
#df_iss
#df_sl = pd.DataFrame(location_sl)
#df_sl = df_sl[['name', 'oname', 'lat', 'lng']]
#df_sl = df_sl.sort_values(['lat', 'lng'] , ascending=True)
#df_sl = df_sl[df_sl['oname'] != 'noname']

for i in iss_location:
    
    if i['latitude']==28.4556:
        print(f'ISS Not Clear')
    if i['longitude']==-80.5278:
        print(f'ISS Not Clear')
    else:
        print(f'ISS Clear')
        break
        
for i in location_sl:
    
    if i['lat']==28.4556:
        print(f'SL Not Clear')
    if i['lng']==-80.5278:
        print(f'SL Not Clear')
    else:
        print(f'SL Clear')
        break
print(df_iss, df_sl)

0
ISS Clear
SL Clear
   longitude latitude
0  -170.6578  47.8607        name          oname      lat       lng
0       2-A  STARLINK 1007   3.0644  -63.2595
1       2-B  STARLINK 1008 -48.5021   61.3151
2       2-C  STARLINK 1009  -7.4826  113.4023
3       2-D  STARLINK 1010 -49.1646   -1.0200
4       2-E  STARLINK 1011 -52.2442   34.0141
...     ...            ...      ...       ...
3834  79-AZ  STARLINK-5758  36.5817  -52.7400
3835  79-BA  STARLINK-5793  19.1059  -83.2979
3836  79-BB  STARLINK-5795  36.9453  -51.7414
3837  79-BC  STARLINK-5798  37.0559  -51.4371
3838  79-BD  STARLINK-5800  37.2914  -50.7614

[3839 rows x 4 columns]


In [ ]:
location_sl = requests.get(url2).json()['sats'][1:]
location_sl[0]["lat"]

In [ ]:
df = pd.read_csv("Falcon_9_and_Falcon_Heavy_launches.csv")
df.columns
df = df[['Date', 'Time', 'Version, Booster', 'Launch_Site', 'Payload', 
        'Payload_Mass', 'Orbit', 'Customer', 'Launch_Outcome', 'Booster_landing']]
df

In [ ]:
f9_filter = df['Version, Booster'].str.contains('F9 B5')
df_f9b5 = df[f9_filter]
df_f9b5

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(18,7))
df_f9b5['Booster_landing'].value_counts().plot(kind="bar", ax=axs[1])
axs[1].set_title("Booster landing")
df_f9b5['Launch_Outcome'].value_counts().plot(kind="bar", ax=axs[0])
axs[0].set_title("Launch outcome")
plt.show()

In [ ]:
df_f9b5['Orbit'].mode()

In [ ]:
df_f9b5['Orbit'].value_counts().plot(kind="bar", figsize=(18,7))

In [ ]:
df_f9b5['Customer'].mode()

In [ ]:
df_f9b5['Customer'].value_counts().plot(kind="bar", figsize=(18,7))

In [ ]:
sns.lineplot(df_f9b5.Date)

In [ ]:
df_f9b5['Launch_Site'].unique()

In [ ]:
sns.histplot(y=df_f9b5.Launch_Site, x=df_f9b5.Booster_landing, palette="magma")


In [ ]:
nasa_filter = df['Customer'].str.contains('NASA')
df_f9b5_nasa = df_f9b5[nasa_filter]
df_f9b5_nasa

In [ ]:
#df_f9b5_nasa['Booster_landing'].value_counts().plot(kind="bar")
#df_f9b5_nasa['Launch_Outcome'].value_counts().plot(kind="bar")

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(18,7))
df_f9b5_nasa['Booster_landing'].value_counts().plot(kind="bar", ax=axs[1])
axs[1].set_title("Booster landing")
df_f9b5_nasa['Launch_Outcome'].value_counts().plot(kind="bar", ax=axs[0])
axs[0].set_title("Launch outcome")

plt.show()